# Kaggle Competition: Toxic Comment Classification

This Jupyter Notebook outlines the approach to build a multi-headed model for the Kaggle competition, which involves predicting different types of comment toxicity.

## Import Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import numpy as np
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Load Data

In [ ]:
train_path = 'train.csv'
test_path = 'test.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Distribution of Toxicity Classes

The following bar chart shows the distribution of different types of toxicity in the dataset. This visualization helps in understanding the prevalence of each type of toxicity within the comments. Such insights are crucial for tailoring the model to better recognize and differentiate between these types.

In [ ]:
# Counting occurrences for each type of toxicity
counts = train_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()

# Creating a bar chart
plt.figure(figsize=(10, 6))
counts.plot(kind='bar')
plt.title('Distribution of Toxicity Classes')  # Title of the plot
plt.xlabel('Toxicity Type')                    # Label for the X-axis
plt.ylabel('Number of Occurrences')            # Label for the Y-axis
plt.xticks(rotation=45)                        # Rotate labels on X-axis for better readability
plt.show()                                     # Display the plot

## Data Preprocessing

In [ ]:
# Download stopwords and wordnet from NLTK
nltk.download('stopwords')
nltk.download('wordnet')
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove special characters and punctuation
    text = re.sub(r'\\W+', ' ', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    return text

# Clean the text data
train_data['comment_text'] = train_data['comment_text'].apply(clean_text)

## Feature Engineering using TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_data['comment_text'])

## Model Building

In [ ]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y_full = train_data[labels]

# Split data into training and validation sets
X_train, X_val, y_train_split, y_val = train_test_split(X_train_tfidf, y_full, test_size=0.2, random_state=42)

# Initialize the model
model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))

## Model Training and Validation

In [ ]:
# Train the model
model.fit(X_train, y_train_split)

# Predict on the validation set
y_pred = model.predict_proba(X_val)

# Calculate ROC AUC score
roc_auc_scores = [roc_auc_score(y_val[label], y_pred[:, i]) for i, label in enumerate(labels)]
mean_roc_auc = np.mean(roc_auc_scores)
print('Mean Column-wise ROC AUC:', mean_roc_auc)

## Hyperparameter Tuning

Hyperparameter tuning involves adjusting the parameters of the machine learning model to improve its performance. For Logistic Regression, a key hyperparameter to tune is the regularization strength (C). Regularization can help prevent overfitting. A lower value of C specifies stronger regularization.

We will use Grid Search technique for hyperparameter tuning

In [ ]:
# Define the parameter grid
param_grid = {'estimator__C': [0.01, 0.1, 1, 10, 100]}

# Grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train_split)

# Best parameters
print("Best parameters:", grid_search.best_params_)

## Final Model Training and Predictions

Train the model on the entire dataset and make predictions on the test dataset.

In [ ]:
# Retrain the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train_tfidf, y_full)

# Preprocess the test data
test_data['comment_text'] = test_data['comment_text'].apply(clean_text)
X_test_tfidf = tfidf_vectorizer.transform(test_data['comment_text'])

# Make predictions
test_predictions = best_model.predict_proba(X_test_tfidf)


## Preparing Submission File

Format the predictions as required for the competition submission

In [ ]:
# Create a submission dataframe
submission = pd.DataFrame(test_predictions, columns=labels)
submission['id'] = test_data['id']
submission = submission[['id'] + labels]

# Save the submission file
submission.to_csv('submission.csv', index=False)